To execute this notebook, you need to either
 - Download a pre-trained model
 - Train an example model by excecuting the model_training_tutorial.ipynb notebook

In this tutorial we will learn to:
- Load a previously trained model
- Extract DeepPrint features from fingerprint images
- Evaluate the performance of the extracted fixed-length representations

## Embedding extraction
After training the model, we can extract the DeepPrint features for the fingerprint images. This is done by calling the `extract` method of the `DeepPrintExtractor` class.

### Imports



In [28]:
import os

from flx.extractor.fixed_length_extractor import get_DeepPrint_Tex, get_DeepPrint_TexMinu, DeepPrintExtractor

# Dimension and number of training subjects must be known to load the pre-trained model

# To load the pre-trained model parameters use num_training_subjects=8000
extractor: DeepPrintExtractor = get_DeepPrint_TexMinu(num_training_subjects=8000, num_dims=256)

# To load the pre-trained model parameters use
MODEL_DIR: str = os.path.abspath("/home/mt0/22CS60R42/fixed-length-fingerprint-extractors/notebooks/DeepPrint_TexMinu_512") # Path to the directory containing the model parameters
extractor.load_best_model(MODEL_DIR)

Loaded best model from /home/mt0/22CS60R42/fixed-length-fingerprint-extractors/notebooks/DeepPrint_TexMinu_512/best_model.pyt


### Extaction of Texture Embeddings and Minutiae Embeddings
Now we need to specify the dataset, for which we want to extract the embeddings

In [2]:
import os

from flx.data.dataset import *
from flx.data.image_loader import SFingeLoader
from flx.data.transformed_image_loader import TransformedImageLoader
from flx.image_processing.binarization import LazilyAllocatedBinarizer
from flx.data.image_helpers import pad_and_resize_to_deepprint_input_size

# NOTE: If this does not work, enter the absolute path to the notebooks/example-dataset directory here! 
DATASET_PATH: str = os.path.abspath("/home/mt0/22CS60R42/fixed-length-fingerprint-extractors/notebooks/example-dataset-roi-images")

# We will use the SFingeLoader to load the images from the dataset
image_loader = TransformedImageLoader(
        images=SFingeLoader(DATASET_PATH),
        poses=None,
        transforms=[
            LazilyAllocatedBinarizer(5.0),
            pad_and_resize_to_deepprint_input_size,
        ],
    )

image_dataset: Dataset = Dataset(image_loader, image_loader.ids)

# The second value is for the minutiae branch, which we do not have in this example
texture_embeddings, minutia_embeddings = extractor.extract(image_dataset)

Created IdentifierSet with 6 subjects and a total of 60 samples.


  0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:26<00:00, 13.30s/it]


In [3]:
from flx.benchmarks.matchers import CosineSimilarityMatcher
from flx.data.embedding_loader import EmbeddingLoader

# We concatenate texture and minutia embedding vectors
embeddings = EmbeddingLoader.combine(texture_embeddings, minutia_embeddings)


Created IdentifierSet with 6 subjects and a total of 60 samples.
Created IdentifierSet with 6 subjects and a total of 60 samples.
Created IdentifierSet with 6 subjects and a total of 60 samples.


### Core Point Extraction

In [ ]:
# import struct
# import numpy as np

# def write_to_file(name, data, siz):
#     f = open(name, 'wb')
#     s = struct.pack('i' * len(siz), *siz)
#     f.write(s)
#     s = struct.pack('f' * len(data), *data)
#     f.write(s)
#     f.close()

# # Assuming texture_embeddings is a 2D array of shape (60, 512)
# texture_embeddings = embeddings  # Your 2D array

# # Normalize texture_embeddings
# texture_embeddings_normalized = texture_embeddings / np.linalg.norm(texture_embeddings, ord=2, axis=1, keepdims=True)

# precision = 125

# # Round the normalized texture_embeddings for precision
# d1 = np.round(texture_embeddings_normalized * precision)
# d2 = np.round(texture_embeddings_normalized * precision)

# # Save in 1-to-1 format
# d1_flattened = d1.flatten()
# d2_flattened = d2.flatten()

# size1 = [60, 512]  # Assuming 60 one-dimensional arrays of 512 dimensions
# size2 = [60, 512]

# write_to_file('probe-1-to-1.bin', d2_flattened, size2)
# write_to_file('gallery-1-to-1.bin', d1_flattened, size1)

# # Save in 1-to-n format
# d1_transposed = d1.T.flatten()  # Transpose d1 before flattening
# d2_transposed = d2.T.flatten()  # Transpose d2 before flattening

# size1 = [512, 60]  # Transpose of size for 1-to-n format
# size2 = [512, 60]

# write_to_file('probe-1-to-n.bin', d2_transposed, size2)
# write_to_file('gallery-1-to-n.bin', d1_transposed, size1)


Waking algo for core to bianry using gray code

In [90]:
import os

def gray_code(n):
    return n ^ (n >> 1)

def convert_to_gray_code(x, y):
    gray_x = gray_code(int(x))
    gray_y = gray_code(int(y))
    return bin(gray_x)[2:], bin(gray_y)[2:]

def pad_binary(binary, length):
    return binary.zfill(length)

def process_txt_file(txt_file, output_folder):
    with open(txt_file, 'r') as f:
        lines = f.readlines()
        x, y = map(float, lines[0].split())
        
        gray_x, gray_y = convert_to_gray_code(x, y)
        
        concatenated_binary = gray_x + gray_y
        
        max_length = len(concatenated_binary)
        
        padded_binary = pad_binary(concatenated_binary, max_length)
        
        output_file = os.path.join(output_folder, os.path.splitext(os.path.basename(txt_file))[0] + '_result.txt')
        with open(output_file, 'w') as out_f:
            out_f.write(padded_binary)

def process_folder(input_folder, output_folder):
    # Create output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for filename in os.listdir(input_folder):
        if filename.endswith('.txt'):
            process_txt_file(os.path.join(input_folder, filename), output_folder)

# Specify input and output folders
input_folder = '/home/mt0/22CS60R42/fixed-length-fingerprint-extractors/notebooks/walking_core_points'
output_folder = '/home/mt0/22CS60R42/fixed-length-fingerprint-extractors/notebooks/walking_binary_core'

# Process all .txt files in the input folder
process_folder(input_folder, output_folder)


For Fingerflow convert core to binary using xor 


In [ ]:
import os

def convert_to_binary(x, y):
    binary_x = bin(int(x))[2:]
    binary_y = bin(int(y))[2:]
    return binary_x, binary_y

def pad_binary(binary, length):
    return binary.zfill(length)

def xor_binary(binary1, binary2):
    return bin(int(binary1, 2) ^ int(binary2, 2))[2:]

def process_txt_file(txt_file):
    with open(txt_file, 'r') as f:
        lines = f.readlines()
        x1, y1 = map(float, lines[0].split())
        x2, y2 = map(float, lines[1].split())
        
        binary_x1, binary_y1 = convert_to_binary(x1, y1)
        binary_x2, binary_y2 = convert_to_binary(x2, y2)
        
        s1 = binary_x1 + binary_y1
        s2 = binary_x2 + binary_y2
        
        max_length = max(len(s1), len(s2))
        s1 = pad_binary(s1, max_length)
        s2 = pad_binary(s2, max_length)
        
        result = xor_binary(s1, s2)
        
        output_file = os.path.splitext(txt_file)[0] + '_result.txt'
        with open(output_file, 'w') as out_f:
            out_f.write(result)

def process_folder(input_folder):
    for filename in os.listdir(input_folder):
        if filename.endswith('.txt'):
            process_txt_file(os.path.join(input_folder, filename))

# Specify input folder containing .txt files
input_folder = '/home/mt0/22CS60R42/fixed-length-fingerprint-extractors/notebooks/fixedlength-core_points2'

# Process all .txt files in the folder
process_folder(input_folder)


Random projection and all

In [3]:
# import numpy as np
# from sklearn.decomposition import PCA

# # Define z-score normalization function
# def z_score_normalize(data):
#     mean_val = np.mean(data)
#     std_dev = np.std(data)
#     normalized_data = (data - mean_val) / std_dev
#     return normalized_data

# # Define function to apply median filter
# def median_filter(data):
#     # Apply median filter to the entire array
#     filtered_data = np.median(data)
#     print(filtered_data)
#     return filtered_data

# # Define function to apply random projection
# def random_projection(data, n_components):
#     # For 1D arrays, random projection doesn't make sense, but you can just return the original array
#     return data

# # Define function to convert normalized array to binary array
# def to_binary_array(data, threshold):
#     return (data >= threshold).astype(int)

# # Choose normalization method and threshold
# normalize_method = z_score_normalize  # Choose either min_max_scale or z_score_normalize
# threshold = 0.3

# # List to store binary arrays
# binary_arrays = []

# # Normalize, filter, and project each array
# for array in texture_embeddings._array:
#     # Normalize data
#     normalized_array = normalize_method(array)
    
#     # Apply median filter
#     filtered_array = median_filter(normalized_array)
    
#     # # Apply random projection
#     # projected_array = random_projection(filtered_array, n_components=10)  # Adjust n_components as needed
    
#     # Convert to binary array
#     binary_array = to_binary_array(filtered_array, threshold)
    
#     # Append to list
#     binary_arrays.append(binary_array)

# # Print binary arrays
# for i, binary_array in enumerate(binary_arrays):
#     print(f"Binary array for array {i}:")
#     print(binary_array)


Hardcored Core points

In [ ]:
# import numpy as np

# # Define z-score normalization function
# def z_score_normalize(data):
#     mean_val = np.mean(data)
#     std_dev = np.std(data)
#     normalized_data = (data - mean_val) / std_dev
#     return normalized_data

# # Define function to convert normalized array to binary array
# def to_binary_array(data, threshold):
#     return (data >= threshold).astype(int)

# # Choose normalization method and threshold
# normalize_method = z_score_normalize  # Choose either min_max_scale or z_score_normalize
# threshold = 0.5

# # List to store binary arrays
# binary_arrays = []

# # Normalize and convert each array to binary
# for array in texture_embeddings._array:
#     # Sort the array (not sure if this step is necessary based on your code)
#     sorted(array)

#     # Normalize the array
#     normalized_array = normalize_method(array)
    
#     # Convert the normalized array to binary using the threshold
#     binary_array = to_binary_array(normalized_array, threshold)
    
#     # Append the binary array to the list
#     binary_arrays.append(binary_array)

# # Generate the permutation pattern from core points (assuming core_points is available)
# core_points = [0, 1, 0, 1]  # Example core points as a binary array

# # Function to permute binary array in intervals using a pattern
# def permute_binary_array_in_intervals(binary_array, pattern):
#     permuted_array = []
#     interval_length = len(binary_array) // len(pattern)
#     for i in range(0, len(binary_array), interval_length):
#         segment = binary_array[i:i+interval_length]
#         permuted_segment = [segment[j] for j in pattern]
#         permuted_array.extend(permuted_segment)
#     return np.array(permuted_array)

# # Permute all binary arrays using the generated pattern
# permuted_arrays = []
# for binary_array in binary_arrays:
#     permuted_binary_array = permute_binary_array_in_intervals(binary_array, core_points)
#     permuted_arrays.append(permuted_binary_array)


# # Print the permuted binary arrays
# for i, permuted_binary_array in enumerate(permuted_arrays):
#     print(f"Permuted binary array for array {i}:")
#     print(permuted_binary_array)


#### Get the core points into an array

In [15]:
# Folder containing core point files
core_points_folder = "/home/mt0/22CS60R42/fixed-length-fingerprint-extractors/notebooks/walking_binary_core"
#TODO: convert each to a class
class fingerprint():  
    def __init__(self, id='1_1',
        texture_feature="",
        core_point="",
        mintutiae=""):
        self.id = id
        self.texture_feature = texture_feature
        self.core_point = core_point
        self.mintutiae = mintutiae
# get_core = f'core/{id}.txt'
# get_texture = f'texture/{id}.txt'
core_point_array = []
# Permute all binary arrays using the core points from files in the folder
for core_points_file in sorted(os.listdir(core_points_folder)):
    # Read core points from the current file
    with open(os.path.join(core_points_folder, core_points_file), "r") as f:
        core_points_str = f.read().strip()

    # Convert the core points string to a list of integers
    core_points = [int(bit) for bit in core_points_str]
    core_point_array.append(core_points)
# print(len(core_point_array))

### Permutations of binary code
This is the permutation with core as binary value seed

#### Texture to Binary array

In [4]:
import numpy as np
import os

# Define z-score normalization function
def z_score_normalize(data):
    mean_val = np.mean(data)
    std_dev = np.std(data)
    normalized_data = (data - mean_val) / std_dev
    return normalized_data

# Define function to convert normalized array to binary array
def to_binary_array(data, threshold):
    return (data >= threshold).astype(int)

# Choose normalization method and threshold
normalize_method = z_score_normalize  # Choose either min_max_scale or z_score_normalize
threshold = 0.3

# List to store binary arrays
binary_arrays = []

# Normalize and convert each array to binary
for array in texture_embeddings._array:
    # Sort the array (not sure if this step is necessary based on your code)
    sorted(array)

    # Normalize the array
    normalized_array = normalize_method(array)
    
    # Convert the normalized array to binary using the threshold
    binary_array = to_binary_array(normalized_array, threshold)
    
    # Append the binary array to the list
    binary_arrays.append(binary_array)
len(binary_arrays)

60

#### To compute the permuation of binary array based on the core points

In [ ]:
### DONT RUN PERMUTATION
## Function to permute binary array in intervals using the pattern
# def permute_binary_array_in_intervals(binary_array, pattern):
#     permuted_array_temp = []
#     interval_length = len(binary_array) // len(pattern)
#     for i in range(0, len(binary_array), interval_length):
#         segment = binary_array[i:i+interval_length]
#         permuted_segment = [segment[j] for j in pattern]
#         permuted_array_temp.extend(permuted_segment)
#     return np.array(permuted_array_temp)

# permuted_array = []
# for binary_array, core_point in zip(binary_arrays, core_point_array):
#     # Permute the binary array using the read core points
#     permuted_binary_array = permute_binary_array_in_intervals(binary_array, core_point)
#     permuted_array.append(permuted_binary_array)
# len(permuted_array)
# # Print the permuted binary arrays
# # for i, permuted_binary_array in enumerate(permuted_array):
#     # print(f"Permuted binary array for array {i}:")
#     # print(permuted_binary_array.shape)


In [17]:
permuted_arrays = binary_arrays[:]

This is for tensors to normalise and with threshold get binary array

In [ ]:
# import numpy as np

# # Define z-score normalization function
# def z_score_normalize(data):
#     mean_val = np.mean(data)
#     std_dev = np.std(data)
#     normalized_data = (data - mean_val) / std_dev
#     return normalized_data

# # Define function to convert normalized array to binary array
# def to_binary_array(data, threshold):
#     return (data >= threshold).astype(int)

# # Choose normalization method and threshold
# normalize_method = z_score_normalize  # Choose either min_max_scale or z_score_normalize
# threshold = 0.3

# # List to store binary arrays
# binary_arrays = []

# # Normalize and convert each array to binary
# for array in embeddings._array:
#     sorted(array)

#     normalized_array = normalize_method(array)
#     binary_array = to_binary_array(normalized_array, threshold)
#     binary_arrays.append(binary_array)

# # Print binary arrays
# for i, binary_array in enumerate(binary_arrays):
#     print(f"Binary array for array {i}:")
#     print(binary_array)

# import numpy as np

# # Function to calculate Hamming distance between two binary arrays
# def hamming_distance(array1, array2):
#     return np.sum(array1 != array2)

# # Initialize lists to store binary arrays for each person
# binary_arrays_same_person = []
# binary_arrays_diff_person = []

# # Normalize and convert each array to binary
# for array in minutia_embeddings._array:
#     # Truncate the array to the first 8 bits
#     truncated_array = array[:8]
    
#     # Convert the truncated array to binary
#     binary_array = to_binary_array(truncated_array, threshold)
    
#     # Append to appropriate list based on person index
#     person_index = i % 10  # Assuming each person has 10 impressions
#     if person_index < 10:
#         binary_arrays_same_person.append(binary_array)
#     else:
#         binary_arrays_diff_person.append(binary_array)

# # Calculate Hamming distance for same person pairs
# print("Hamming distances for same person pairs:")
# for i in range(len(binary_arrays_same_person)):
#     for j in range(i + 1, len(binary_arrays_same_person)):
#         distance = hamming_distance(binary_arrays_same_person[i], binary_arrays_same_person[j])
#         print(f"Impression {i} vs Impression {j}: {distance}")

# # Calculate Hamming distance for different person pairs
# print("\nHamming distances for different person pairs:")
# for i in range(len(binary_arrays_diff_person)):
#     for j in range(len(binary_arrays_diff_person)):
#         distance = hamming_distance(binary_arrays_diff_person[i], binary_arrays_diff_person[j])
#         print(f"Impression {i} vs Impression {j}: {distance}")

For permutation one

### Hamming Distance of Permuted Array

In [ ]:
# Function to calculate Hamming distance between two binary arrays
def hamming_distance(array1, array2):
    return np.sum(array1 != array2)

# Initialize lists to store binary arrays for each person
binary_arrays_same_person = []
binary_arrays_diff_person = []


# Populate binary arrays lists
for i, permuted_binary_array in enumerate(permuted_arrays):
    # Append to appropriate list based on person index
    person_index = i // 10  # Assuming each person has 10 impressions
    if person_index < 6:
        binary_arrays_same_person.append(permuted_binary_array)
    else:
        binary_arrays_diff_person.append(permuted_binary_array)

# Calculate Hamming distance for same person pairs
print("Hamming distances for same person pairs:")
for i in range(len(binary_arrays_same_person)):
    for j in range(i + 1, len(binary_arrays_same_person)):
        distance = hamming_distance(binary_arrays_same_person[i], binary_arrays_same_person[j])
        print(f"Impression {i} vs Impression {j}: {distance}")

# Calculate Hamming distance for different person pairs
print("\nHamming distances for different person pairs:")
for i in range(len(binary_arrays_same_person)):
    for j in range(len(binary_arrays_diff_person)):
        distance = hamming_distance(binary_arrays_same_person[i], binary_arrays_diff_person[j])
        print(f"Impression {i} (Same Person) vs Impression {j} (Different Person): {distance}")


Hamming distance code


In [21]:
import numpy as np

# Function to calculate Hamming distance between two binary arrays
def hamming_distance(array1, array2):
    return np.sum(array1 != array2)

# Initialize lists to store binary arrays for each person
binary_arrays_same_person = []
binary_arrays_diff_person = []

# Normalize and convert each array to binary
for array in embeddings._array:
    normalized_array = normalize_method(array)
    binary_array = to_binary_array(normalized_array, threshold)
    
    # Append to appropriate list based on person index
    person_index = i % 10  # Assuming each person has 10 impressions
    if person_index < 10:
        binary_arrays_same_person.append(binary_array)
    else:
        binary_arrays_diff_person.append(binary_array)

# Calculate Hamming distance for same person pairs
print("Hamming distances for same person pairs:")
for i in range(len(binary_arrays_same_person)):
    for j in range(i + 1, len(binary_arrays_same_person)):
        distance = hamming_distance(binary_arrays_same_person[i], binary_arrays_same_person[j])
        print(f"Impression {i} vs Impression {j}: {distance}")

# Calculate Hamming distance for different person pairs
print("\nHamming distances for different person pairs:")
for i in range(len(binary_arrays_diff_person)):
    for j in range(len(binary_arrays_diff_person)):
        distance = hamming_distance(binary_arrays_diff_person[i], binary_arrays_diff_person[j])
        print(f"Impression {i} vs Impression {j}: {distance}")


Hamming distances for same person pairs:
Impression 0 vs Impression 1: 28
Impression 0 vs Impression 2: 31
Impression 0 vs Impression 3: 33
Impression 0 vs Impression 4: 25
Impression 0 vs Impression 5: 53
Impression 0 vs Impression 6: 39
Impression 0 vs Impression 7: 33
Impression 0 vs Impression 8: 27
Impression 0 vs Impression 9: 27
Impression 0 vs Impression 10: 190
Impression 0 vs Impression 11: 184
Impression 0 vs Impression 12: 187
Impression 0 vs Impression 13: 189
Impression 0 vs Impression 14: 189
Impression 0 vs Impression 15: 185
Impression 0 vs Impression 16: 189
Impression 0 vs Impression 17: 187
Impression 0 vs Impression 18: 192
Impression 0 vs Impression 19: 192
Impression 0 vs Impression 20: 216
Impression 0 vs Impression 21: 216
Impression 0 vs Impression 22: 218
Impression 0 vs Impression 23: 227
Impression 0 vs Impression 24: 218
Impression 0 vs Impression 25: 224
Impression 0 vs Impression 26: 220
Impression 0 vs Impression 27: 217
Impression 0 vs Impression 28: 2

In [23]:
def transform(binary, pattern):
    # Convert binary and pattern strings into lists for easier manipulation
    binary_list = list(binary)
    pattern_list = list(pattern)
    
    # Get the lengths of binary and pattern lists
    n = len(binary_list)
    m = len(pattern_list)

    # Initialize variables
    result = []         # To store the final result
    skipped_bits = []   # To store bits skipped based on the pattern
    j = 0               # Pointer to track the current position in the pattern

    # Iterate through each bit in the binary string
    for i in range(n):
        if pattern_list[j] == '1':  
            # If the current pattern bit is '1', add the corresponding binary bit to the result
            result.append(str(binary_list[i]))  # Ensure this is a string
        else:
            # If the current pattern bit is '0', skip the binary bit and store it in skipped_bits
            skipped_bits.append(str(binary_list[i]))  # Ensure this is a string
        
        # Move to the next pattern bit
        j += 1  
        
        # Reset the pattern pointer when the end of the pattern is reached
        if j == m:
            j = 0
            
    # Append all skipped bits to the result at the end
    result.extend(str(bit) for bit in skipped_bits)  # Convert skipped bits to strings
    
    # Convert the result list back into a string and return it
    return ''.join(result)

# Function to handle lists of binary and pattern strings
def transform_all(binaries, patterns):
    # Apply the transform function to each pair of binary and pattern
    results = [
        transform(binary, pattern) 
        for binary, pattern in zip(binaries, patterns)
    ]
    return results

# Example input
binary = binary_arrays
pattern = core_point_array

# Apply the transformation to each binary-pattern pair
result = transform_all(binary, pattern)

# Print the results
# print(len(result))
for x in result:
    print("OUTPUT",x)

def hamming_dist(s1, s2):
    if len(s1) != len(s2):
        return -1
    return sum(c1 != c2 for c1, c2 in zip(s1, s2))

num_rows = 10
num_cols = 10
hamming_distance_matrix = np.zeros((num_rows, num_cols), dtype=int)

for i in range(num_rows):
    x = result[i]
    for j in range(num_cols):
        y = result[j]
        hamming_distance_matrix[i, j] = hamming_dist(x, y)

OUTPUT 0111011000001101000100001101101000010000010001100000111010100001101011111111000110000010011101010011100001010111000000001001110011010010100110011110001110010001000001001010000101100010110010010011000111100000011100110110000101011000011000100100000001000010
OUTPUT 0111011000001000000000001101101000010000010001100000111010100001101011111101000110000010011100010011100101000111000000001001010011010010100010011010001110010001000001001000000101100010110010010011000111100000011100110110000101011000010000100100000101010010
OUTPUT 0111011000000001000000001101101000010000010001100000101010100001101011111101000110000010011100010011100001010111000000001001010011010010100010011010001110000001000001001010000101100010110010010011000111100000011100110110000101011000010000100100000101000010
OUTPUT 011101110000000100010000110110100001000001000110000010101010000111101111110100011000001000110001001110000101011100000000100101001101001010001001111000111000000100000100100000010110001011001001001100011

In [27]:
hamming_distance_matrix

array([[ 0, 14, 12, 14, 12, 36, 17, 11, 11, 12],
       [14,  0,  8, 12, 18, 40, 21,  7,  7,  8],
       [12,  8,  0,  6, 10, 32, 13,  7,  3, 10],
       [14, 12,  6,  0, 12, 30, 13, 11,  7, 12],
       [12, 18, 10, 12,  0, 24,  9, 13, 13, 16],
       [36, 40, 32, 30, 24,  0, 21, 33, 35, 38],
       [17, 21, 13, 13,  9, 21,  0, 16, 16, 19],
       [11,  7,  7, 11, 13, 33, 16,  0,  6,  5],
       [11,  7,  3,  7, 13, 35, 16,  6,  0,  9],
       [12,  8, 10, 12, 16, 38, 19,  5,  9,  0]])

## Benchmarking

To evaluate the embeddings, we want to run a benchmark on them. For this, we must first specify the type of benchmark, and which comparisons should be run.

In [18]:
from flx.scripts.generate_benchmarks import create_verification_benchmark

NUM_IMPRESSIONS_PER_SUBJECT = 10
benchmark = create_verification_benchmark(
    #subjects=list(range(100, image_dataset.num_subjects + 100)),
    subjects=list(range(image_dataset.num_subjects)),
    impressions_per_subject=list(range(NUM_IMPRESSIONS_PER_SUBJECT))
)

  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:00<00:00, 17939.71it/s]


Now we can run the benchmark. To do this, we must first specify the matcher (in our case cosine similarity of the embeddings)

In [11]:
from flx.benchmarks.matchers import CosineSimilarityMatcher
from flx.data.embedding_loader import EmbeddingLoader

# We concatenate texture and minutia embedding vectors
embeddings = EmbeddingLoader.combine(texture_embeddings, minutia_embeddings)
matcher = CosineSimilarityMatcher(EmbeddingLoader.combine(texture_embeddings, minutia_embeddings))

results = benchmark.run(matcher)

print(f"Equal-Error-Rate: {results.get_equal_error_rate()}")

Created IdentifierSet with 6 subjects and a total of 60 samples.
Created IdentifierSet with 6 subjects and a total of 60 samples.
Created IdentifierSet with 6 subjects and a total of 60 samples.
Created IdentifierSet with 6 subjects and a total of 60 samples.
Created IdentifierSet with 6 subjects and a total of 60 samples.
Created IdentifierSet with 6 subjects and a total of 60 samples.


NameError: name 'benchmark' is not defined

To visualize the results, we can plot a DET curve. (Do not wonder if it is empty, probably the model is not trained enough. Take a look at the EER instead.)

In [10]:
from flx.visualization.plot_DET_curve import plot_verification_results

figure_path = "DET_curve"

# Lists are used to allow for multiple models to be plotted in the same figure
plot_verification_results(figure_path, results=[results], model_labels=["DeepPrint_TexMinu"], plot_title="example-dataset - verification")

/home/mt0/22CS60R42/.anaconda3/lib/python3.11/site-packages/matplotlib/pyplot.py:2201: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if fignum is False or fignum is 0:
/home/mt0/22CS60R42/.anaconda3/lib/python3.11/site-packages/matplotlib/pyplot.py:2201: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if fignum is False or fignum is 0:
/home/mt0/22CS60R42/.anaconda3/lib/python3.11/site-packages/matplotlib/pyplot.py:2201: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if fignum is False or fignum is 0:
/home/mt0/22CS60R42/.anaconda3/lib/python3.11/site-packages/matplotlib/pyplot.py:2201: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if fignum is False or fignum is 0:


SystemError: PY_SSIZE_T_CLEAN macro must be defined for '#' formats

SystemError: PY_SSIZE_T_CLEAN macro must be defined for '#' formats

<Figure size 640x480 with 1 Axes>